# Now we build a handwriting recognition app to apply our model for use

In [2]:
import gradio as gr
import tensorflow as tf
import os

## Load the model 

In [3]:
model_url_xwzj = '/Users/derryzrli/Downloads/data_dsi_capstone/traditional_chinese_training_data_xwzj/CNN_model_xwzj.h5'

In [5]:
model = tf.keras.models.load_model(model_url_xwzj)

2022-06-12 18:22:33.509485: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
class_map = {'境': 0, '學': 1, '止': 2, '無': 3}
class_map = {val:key for key, val in class_map.items()}
class_map

{0: '境', 1: '學', 2: '止', 3: '無'}

In [7]:
from PIL import Image
import numpy as np

img = Image.open('/Users/derryzrli/Downloads/data_dsi_capstone/traditional_chinese_training_data/止/止_11.png')
img = np.array(img)/255.
img = tf.image.resize(img, [50,50])
img = img.numpy()
img = img.reshape(1 ,50, 50, 3) # 1 observation, 50 px high, 50 px wide, 3 depth
pred = model.predict(img)
values, indices = tf.math.top_k(pred[0], 3)
confidences = {class_map[i.numpy()]: float(v.numpy()) for i, v in zip(indices, values)}
confidences

2022-06-12 18:22:42.465404: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


{'止': 0.8266451358795166, '無': 0.15484075248241425, '境': 0.01700018160045147}

## Make Classify Function

In [8]:
def classify(img):
    # img = img.reshape(28, 28, 1)
    # x = tf.image.grayscale_to_rgb(tf.convert_to_tensor(img)).numpy()
    x = np.array(img)/255.

    x = tf.image.resize(x, [50,50]).numpy()
    x = x.reshape(1, 50, 50, 3)
    pred = model.predict(x)
    values, indices = tf.math.top_k(pred[0], 3)
    confidences = {class_map[i.numpy()]: float(v.numpy()) for i, v in zip(indices, values)}
    return confidences

## Instantiate Gradio App

In [10]:
gr.Interface(fn=classify, 
             inputs=gr.inputs.Image(),
             outputs=gr.outputs.Label(num_top_classes=3),
             examples=['derry_chinese_handwriting/wu_1.png', 'derry_chinese_handwriting/wu_2.png', 'derry_chinese_handwriting/wu_3.png', 
                      'derry_chinese_handwriting/zhi_1.png', 'derry_chinese_handwriting/zhi_3.png', 'derry_chinese_handwriting/zhi_2.png']).launch(share=True)

/Users/derryzrli/opt/anaconda3/envs/tfm1/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Users/derryzrli/opt/anaconda3/envs/tfm1/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7861/
Running on public URL: https://58017.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f85d81d4760>,
 'http://127.0.0.1:7861/',
 'https://58017.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/Users/derryzrli/opt/anaconda3/envs/tfm1/lib/python3.9/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


## Viola~